In [ ]:
# import the necessary libraries

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Normalization
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
# load the data

df = pd.read_csv("/content/housing.csv", header=None, delim_whitespace = True)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


In [ ]:
# split the data to X and y

y = df[13]
X = df.drop(13, axis=1)

In [ ]:
# split the data to training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
# model

def dense_model():
  input = Input(shape = (13))
  x = Dense(128, activation = 'relu')(input)
  x = Dense(64, activation = 'relu')(x)
  x = Dense(32, activation = 'relu')(x)
  output = Dense(1, activation = 'linear')(x)

  model = tf.keras.models.Model(inputs = input, outputs = output)
  model.compile(loss = 'mse', optimizer = 'adam')

  return model

dense_model().summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense (Dense)               (None, 128)               1792      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 12,161
Trainable params: 12,161
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Wrap keras model with scikit-learn to perform other operations
regressor = KerasRegressor(build_fn=dense_model, epochs=50, batch_size=32, verbose = 0)

# pipeline the wrapped model
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('regressor', regressor),
])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


In [ ]:
# kfold split parameters
kfold = KFold(n_splits=5, shuffle=True)

# kfold cross-validation 
results = cross_val_score(pipe, X_train, y_train, cv=kfold)

print(f"The average mean squared error is { results.mean() } \nThe std is { results.std() }")

The average mean squared error is -15.564438629150391 
The std is 6.195488836981345


In [ ]:
# fit the model
pipe.fit(X_train, y_train)

# make predictions on test data
y_pred = pipe.predict(X_test)

# compute mean squared error
mse_krr = mean_squared_error(y_test, y_pred)
print(f' The mean squared error on test data is {mse_krr}')


 The mean squared error on test data is 10.66330125144064


In [ ]:
%%time

def one_layer_dense_model(units):
  input = Input(shape = (13))
  x = Normalization(axis = -1)(input)
  x = Dense(units, activation = 'relu')(x)
  output = Dense(1, activation = 'linear')(x)

  model = tf.keras.models.Model(inputs = input, outputs = output)
  model.compile(loss = 'mse', optimizer = 'adam')
  return model

regressor2 = KerasRegressor(build_fn=one_layer_dense_model, epochs=100, batch_size=32, verbose = 0)

params={ 'units': [16*i for i in range(26)]  }

gs = GridSearchCV(estimator=regressor2, param_grid = params, cv=5)
gs = gs.fit(X, y)

print(gs.best_score_)
print(gs.best_params_)
#print(f"The average mean squared error is { results.mean() } \nThe std is { results.std() }")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  if sys.path[0] == '':


-38.496948623657225
{'units': 368}
CPU times: user 6min 21s, sys: 23.7 s, total: 6min 45s
Wall time: 6min 15s


Generally, with increasing depth of the neural networks, the required number of epochs to reach a good solution (mse loss of 40 or less), decreased. 

In [ ]:
def arbitary_depth_dense_model(units, depth):
  input = Input(shape = (13))
  x = Normalization(axis = -1)(input)
  for i in range(depth):
    x = Dense(units, activation = 'relu')(x)
  output = Dense(1, activation = 'linear')(x)

  model = tf.keras.models.Model(inputs = input, outputs = output)
  model.compile(loss = 'mse', optimizer = 'adam')
  return model

class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if logs['val_loss'] <= 40 or epoch >= 100:
      self.model.stop_training = True
      print("\n\nEither reached the threshold validation loss of 40 or reached the limit of epochs 100")
      print(f'epoch : {epoch} \nval_loss : {logs["val_loss"]}\n')

        
for depth in range(1, 6):
  model = arbitary_depth_dense_model(32, depth)
  
  model.fit(X_train, y_train, 
            epochs = 100,
            batch_size = 32, 
            validation_data = (X_test, y_test),
            callbacks = [MyCallback()])

Epoch 1/100
13/13 [==============================] - 1s 16ms/step - loss: 3702.6316 - val_loss: 1308.0875
Epoch 2/100
13/13 [==============================] - 0s 4ms/step - loss: 831.4778 - val_loss: 662.5618
Epoch 3/100
13/13 [==============================] - 0s 7ms/step - loss: 574.4820 - val_loss: 431.2414
Epoch 4/100
13/13 [==============================] - 0s 4ms/step - loss: 304.0829 - val_loss: 249.2427
Epoch 5/100
13/13 [==============================] - 0s 6ms/step - loss: 193.5388 - val_loss: 195.5062
Epoch 6/100
13/13 [==============================] - 0s 5ms/step - loss: 136.2961 - val_loss: 159.1164
Epoch 7/100
13/13 [==============================] - 0s 6ms/step - loss: 119.1937 - val_loss: 151.2954
Epoch 8/100
13/13 [==============================] - 0s 5ms/step - loss: 112.0767 - val_loss: 150.0656
Epoch 9/100
13/13 [==============================] - 0s 4ms/step - loss: 107.8226 - val_loss: 146.9340
Epoch 10/100
13/13 [==============================] - 0s 6ms/step - lo